In [3]:
import os
import cv2

path_1 = './img/jjajang'
path_2 = './img/tangsu'
path_t = './data/tangjja'

id = 0

# 파일 이름 일괄 변경
for (path, dirs, files) in os.walk(path_1):
    for filename in files:
        newname = '0_{}.jpg'.format(id)
        id+=1
        os.rename(path_1 + '/' + filename, path_t + '/' + newname)

for (path, dirs, files) in os.walk(path_2):
    for filename in files:
        newname = '1_{}.jpg'.format(id)
        id+=1
        os.rename(path_2 + '/' + filename, path_t + '/' + newname)

In [27]:
# import os
# import cv2

# path_t = './data/tangjja'

# 파일 사이즈 일괄 변경
IMG_SIZE = 32

for filename in os.listdir(path_t):
    try:
        path = path_t + '/' + filename
        img = cv2.imread(path, cv2.IMREAD_COLOR)
        img1 = cv2.resize(img, dsize = (IMG_SIZE, IMG_SIZE), interpolation = cv2.INTER_AREA)
        cv2.imwrite('./data/tangjja_resize/' + '/' + filename, img1)
    except Exception as e:
        print(str(e))

OpenCV(4.5.1) /private/var/folders/nz/vv4_9tw56nv9k3tkvyszvwg80000gn/T/pip-req-build-yvyj7qlp/opencv/modules/imgproc/src/resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.1) /private/var/folders/nz/vv4_9tw56nv9k3tkvyszvwg80000gn/T/pip-req-build-yvyj7qlp/opencv/modules/imgproc/src/resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'resize'



In [3]:
from keras.datasets import cifar10
from PIL import Image
import numpy as np
import os

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

max_num_datas = 100
test_num_datas = 20
num_classes = 4
num_datas_list = np.zeros(num_classes)

img_train = './data/cifar_train/'
img_test = './data/cifar_test/'

id = 0

for x, y in zip(x_train, y_train):
    if np.sum(num_datas_list) > (max_num_datas + test_num_datas)*len(num_datas_list):
        break
    label = y[0]
    if label >= num_classes:
        continue
        
    #download training data
    if num_datas_list[label] < max_num_datas:
        num_datas_list[label] += 1
        
        img_path = os.path.join(img_train, '{}_{}.jpg'.format(label, id))
        id += 1
        img = Image.fromarray(x)
        img.save(img_path)
        
    #download tesing data
    if num_datas_list[label] >= max_num_datas and num_datas_list[label] < (max_num_datas*test_num_datas):
        num_datas_list[label] += 1
        
        img_path = os.path.join(img_test, '{}_{}.jpg'.format(label, id))
        id += 1
        img = Image.fromarray(x)
        img.save(img_path)
        
    if num_datas_list[label] >= max_num_datas + test_num_datas:
        continue

In [1]:
import os
from PIL import Image
import numpy as np
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

data_dir = './data/cifar_train/' # 트레이닝 이미지 저장 폴더
data_dirt = './data/cifar_test/' # 테스트 이미지 저장 폴더

#train = True  # 트레이닝 or 테스트 선택하기
train = False  # 트레이닝 or 테스트 선택하기
model_path = './models/cifar_model' #model 저장경로

# 이미지 폴더로부터 이미지들을 load하고 numpy에 저장한다.
# 이미지 filename이 1_40.jpg 일 때, Lable=1 이다.
def read_data(data_dir):
    datas = []
    labels = []
    fpaths = []
    for fname in os.listdir(data_dir):
        fpath = os.path.join(data_dir, fname)
        fpaths.append(fpath)
        image = Image.open(fpath)
        data = np.array(image) / 255.0
        label = int(fname.split("_")[0])
        datas.append(data)
        labels.append(label)

    datas = np.array(datas)
    labels = np.array(labels)

    print("shape of datas: {}\tshape of labels: {}".format(datas.shape, labels.shape))
    return fpaths, datas, labels

fpaths, datas, labels = read_data(data_dir) #트레이닝 데이터
fpathst, datast, labelst = read_data(data_dirt) #테스트 데이터

# 이미지 class 개수
num_classes = len(set(labels))

# Placeholder에 input데이터와 label를 저장한다.
datas_placeholder = tf.placeholder(tf.float32, [None, 32, 32, 3])
labels_placeholder = tf.placeholder(tf.int32, [None])

# Placeholder에 DropOut 파라미터를 저장한다.트레이닝할 때 0.25，테스트 할 때 0.
dropout_placeholdr = tf.placeholder(tf.float32)

# Conv Layer0: input=32x32x3, filters=20, kernal_size=5，stride=1x1
#conv0 = tf.keras.layers.Conv2D(datas_placeholder, 20, 5, activation=tf.nn.relu)
conv0 = tf.layers.conv2d(datas_placeholder, 20, 5, activation=tf.nn.relu)

print(conv0)
# output size= [(W - F + 2P)/S] +1 = (32-5)/1+1 =28
# Tensor("conv2d/Relu:0", shape=(?, 28, 28, 20), dtype=float32)

# max-pooling0: input=28x28x20, pooling size=2x2，stride=2x2
pool0 = tf.layers.max_pooling2d(conv0, [2, 2], [2, 2])
print(pool0)
# output size= W = [(W - F )/S] +1 = [(28-2)/2]+1 =14
# Tensor("max_pooling2d/MaxPool:0", shape=(?, 14, 14, 20), dtype=float32)

# Conv Layer1: input=14x14x20, filters=40, kernal_size=4，stride=1x1
conv1 = tf.layers.conv2d(pool0, 40, 4, activation=tf.nn.relu)
print(conv1)
# output size= [(W - F + 2P)/S] +1 = (14-4)/1+1 =11
# Tensor("conv2d_1/Relu:0", shape=(?, 11, 11, 40), dtype=float32)

# max-pooling1: input=11x11x40, pooling size=2x2，stride=2x2
pool1 = tf.layers.max_pooling2d(conv1, [2, 2], [2, 2])
print(pool1)
# output size= W = [(W - F )/S] +1 = [(11-2)/2]+1 =5.5
# Tensor("max_pooling2d_1/MaxPool:0", shape=(?, 5, 5, 40), dtype=float32)

# 3-D vector을 1-D vector로 변환한다.
flatten = tf.layers.flatten(pool1)
print(flatten)
# output size=5x5x40=1000
# Tensor("flatten/Reshape:0", shape=(?, 1000), dtype=float32)

# fully-connected layer
fc = tf.layers.dense(flatten, 400, activation=tf.nn.relu)
print(fc)
# Tensor("dense/Relu:0", shape=(?, 400), dtype=float32)

# DropOut를 추가하여 overfitting을 방지한다.
dropout_fc = tf.layers.dropout(fc, dropout_placeholdr)
print(dropout_fc)
# Tensor("dropout/Identity:0", shape=(?, 400), dtype=float32)

# output=4개 class
logits = tf.layers.dense(dropout_fc, num_classes)
print(logits)
#Tensor("dense_1/BiasAdd:0", shape=(?, 4), dtype=float32)

predicted_labels = tf.arg_max(logits, 1)


# loss를 정의한다.
losses = tf.nn.softmax_cross_entropy_with_logits(
    labels=tf.one_hot(labels_placeholder, num_classes), #실제 label
    logits=logits #트레이닝 output label
)

mean_loss = tf.reduce_mean(losses)

# optimizer를 정의한다.
optimizer = tf.train.AdamOptimizer(learning_rate=1e-2).minimize(losses)


saver = tf.train.Saver()

with tf.Session() as sess:

    if train:
        print("training")
        # 트레이닝하기전 initializing 한다.
        sess.run(tf.global_variables_initializer())

        # placeholder에 데이터를 넣는다.
        train_feed_dict = {
            datas_placeholder: datas,
            labels_placeholder: labels,
            dropout_placeholdr: 0.25
        }
        for step in range(1000):
            _, mean_loss_val = sess.run([optimizer, mean_loss], feed_dict=train_feed_dict)

            if step % 10 == 0:
                print("step = {}\tmean loss = {}".format(step, mean_loss_val))
        saver.save(sess, model_path)
        print("training done! saved in {}".format(model_path))
    else:
        print("testing")
        # 트레이닝한 model을 불러온다
        saver.restore(sess, model_path)
        print("from {} loaded".format(model_path))
        # label의 각 의미
        label_name_dict = {
            0: "airplane",
            1: "car",
            2: "bird",
            3: "cat"
        }
        # placeholder에 데이터를 넣는다.
        test_feed_dict = {
            datas_placeholder: datast,
            labels_placeholder: labelst,
            dropout_placeholdr: 0
        }
        predicted_labels_val = sess.run(predicted_labels, feed_dict=test_feed_dict)
        # 테스트결과와 실제결과를 비교한다.
        for fpathst, real_label, predicted_label in zip(fpathst, labelst, predicted_labels_val):
            real_label_name = label_name_dict[real_label]
            predicted_label_name = label_name_dict[predicted_label]
            print("{}\t{} => {}".format(fpathst, real_label_name, predicted_label_name))

Instructions for updating:
non-resource variables are not supported in the long term
shape of datas: (442, 32, 32, 3)	shape of labels: (442,)
shape of datas: (122, 32, 32, 3)	shape of labels: (122,)
Tensor("conv2d/Relu:0", shape=(?, 28, 28, 20), dtype=float32)
Tensor("max_pooling2d/MaxPool:0", shape=(?, 14, 14, 20), dtype=float32)
Tensor("conv2d_1/Relu:0", shape=(?, 11, 11, 40), dtype=float32)
Tensor("max_pooling2d_1/MaxPool:0", shape=(?, 5, 5, 40), dtype=float32)
Tensor("flatten/Reshape:0", shape=(?, 1000), dtype=float32)
Tensor("dense/Relu:0", shape=(?, 400), dtype=float32)
Tensor("dropout/Identity:0", shape=(?, 400), dtype=float32)
Tensor("dense_1/BiasAdd:0", shape=(?, 4), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

testing
INFO:tensorflow:Restoring parameters from ./models/cifar_model


/Users/keunmo/Workspace/projects/chatbot/cbvenv/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/convolutional.py:414: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  warnings.warn('`tf.layers.conv2d` is deprecated and '
/Users/keunmo/Workspace/projects/chatbot/cbvenv/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/Users/keunmo/Workspace/projects/chatbot/cbvenv/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/pooling.py:310: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling2D` instead.
  warnings.warn('`tf.layers.max_pooling2d` is deprecated and '
/Users/keunmo/Workspace/p

from ./models/cifar_model loaded
./data/cifar_test/1_163.jpg	car => car
./data/cifar_test/1_188.jpg	car => car
./data/cifar_test/1_361.jpg	car => car
./data/cifar_test/1_201.jpg	car => car
./data/cifar_test/1_215.jpg	car => car
./data/cifar_test/3_462.jpg	cat => cat
./data/cifar_test/1_374.jpg	car => airplane
./data/cifar_test/1_148.jpg	car => car
./data/cifar_test/0_398.jpg	airplane => airplane
./data/cifar_test/1_410.jpg	car => car
./data/cifar_test/2_458.jpg	bird => car
./data/cifar_test/3_461.jpg	cat => cat
./data/cifar_test/3_475.jpg	cat => cat
./data/cifar_test/3_460.jpg	cat => bird
./data/cifar_test/3_448.jpg	cat => cat
./data/cifar_test/0_206.jpg	airplane => airplane
./data/cifar_test/2_465.jpg	bird => airplane
./data/cifar_test/1_203.jpg	car => car
./data/cifar_test/1_388.jpg	car => car
./data/cifar_test/1_405.jpg	car => car
./data/cifar_test/0_400.jpg	airplane => airplane
./data/cifar_test/0_414.jpg	airplane => cat
./data/cifar_test/1_161.jpg	car => car
./data/cifar_test/1_16

In [1]:
import os
from PIL import Image
import numpy as np
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

data_dir = './data/tangjja_train/' # 트레이닝 이미지 저장 폴더
data_dirt = './data/tangjja_test/' # 테스트 이미지 저장 폴더

#train = True  # 트레이닝 or 테스트 선택하기
train = False  # 트레이닝 or 테스트 선택하기
model_path = './models/tangjja.model' #model 저장경로

# 이미지 폴더로부터 이미지들을 load하고 numpy에 저장한다.
# 이미지 filename이 1_40.jpg 일 때, Lable=1 이다.
def read_data(data_dir):
    datas = []
    labels = []
    fpaths = []
    for fname in os.listdir(data_dir):
        fpath = os.path.join(data_dir, fname)
        fpaths.append(fpath)
        image = Image.open(fpath)
        data = np.array(image) / 255.0
        label = int(fname.split("_")[0])
        datas.append(data)
        labels.append(label)

    datas = np.array(datas)
    labels = np.array(labels)

    print("shape of datas: {}\tshape of labels: {}".format(datas.shape, labels.shape))
    return fpaths, datas, labels

fpaths, datas, labels = read_data(data_dir) #트레이닝 데이터
fpathst, datast, labelst = read_data(data_dirt) #테스트 데이터

# 이미지 class 개수
num_classes = len(set(labels))
print(num_classes)

# Placeholder에 input데이터와 label를 저장한다.
datas_placeholder = tf.placeholder(tf.float32, [None, 32, 32, 3])
labels_placeholder = tf.placeholder(tf.int32, [None])

# Placeholder에 DropOut 파라미터를 저장한다.트레이닝할 때 0.25，테스트 할 때 0.
dropout_placeholdr = tf.placeholder(tf.float32)

# Conv Layer0: input=32x32x3, filters=20, kernal_size=5，stride=1x1
#conv0 = tf.keras.layers.Conv2D(datas_placeholder, 20, 5, activation=tf.nn.relu)
conv0 = tf.layers.conv2d(datas_placeholder, 20, 5, activation=tf.nn.relu)

print(conv0)
# output size= [(W - F + 2P)/S] +1 = (32-5)/1+1 =28
# Tensor("conv2d/Relu:0", shape=(?, 28, 28, 20), dtype=float32)

# max-pooling0: input=28x28x20, pooling size=2x2，stride=2x2
pool0 = tf.layers.max_pooling2d(conv0, [2, 2], [2, 2])
print(pool0)
# output size= W = [(W - F )/S] +1 = [(28-2)/2]+1 =14
# Tensor("max_pooling2d/MaxPool:0", shape=(?, 14, 14, 20), dtype=float32)

# Conv Layer1: input=14x14x20, filters=40, kernal_size=4，stride=1x1
conv1 = tf.layers.conv2d(pool0, 40, 4, activation=tf.nn.relu)
print(conv1)
# output size= [(W - F + 2P)/S] +1 = (14-4)/1+1 =11
# Tensor("conv2d_1/Relu:0", shape=(?, 11, 11, 40), dtype=float32)

# max-pooling1: input=11x11x40, pooling size=2x2，stride=2x2
pool1 = tf.layers.max_pooling2d(conv1, [2, 2], [2, 2])
print(pool1)
# output size= W = [(W - F )/S] +1 = [(11-2)/2]+1 =5.5
# Tensor("max_pooling2d_1/MaxPool:0", shape=(?, 5, 5, 40), dtype=float32)

# 3-D vector을 1-D vector로 변환한다.
flatten = tf.layers.flatten(pool1)
print(flatten)
# output size=5x5x40=1000
# Tensor("flatten/Reshape:0", shape=(?, 1000), dtype=float32)

# fully-connected layer
fc = tf.layers.dense(flatten, 400, activation=tf.nn.relu)
print(fc)
# Tensor("dense/Relu:0", shape=(?, 400), dtype=float32)

# DropOut를 추가하여 overfitting을 방지한다.
dropout_fc = tf.layers.dropout(fc, dropout_placeholdr)
print(dropout_fc)
# Tensor("dropout/Identity:0", shape=(?, 400), dtype=float32)

# output=4개 class
logits = tf.layers.dense(dropout_fc, num_classes)
print(logits)
#Tensor("dense_1/BiasAdd:0", shape=(?, 4), dtype=float32)

predicted_labels = tf.arg_max(logits, 1)


# loss를 정의한다.
losses = tf.nn.softmax_cross_entropy_with_logits(
    labels=tf.one_hot(labels_placeholder, num_classes), #실제 label
    logits=logits #트레이닝 output label
)

mean_loss = tf.reduce_mean(losses)

# optimizer를 정의한다.
optimizer = tf.train.AdamOptimizer(learning_rate=1e-2).minimize(losses)


saver = tf.train.Saver()

with tf.Session() as sess:

    if train:
        print("training")
        # 트레이닝하기전 initializing 한다.
        sess.run(tf.global_variables_initializer())

        # placeholder에 데이터를 넣는다.
        train_feed_dict = {
            datas_placeholder: datas,
            labels_placeholder: labels,
            dropout_placeholdr: 0.25
        }
        for step in range(300):
            _, mean_loss_val = sess.run([optimizer, mean_loss], feed_dict=train_feed_dict)

            if step % 10 == 0:
                print("step = {}\tmean loss = {}".format(step, mean_loss_val))
        saver.save(sess, model_path)
        print("training done! saved in {}".format(model_path))
    else:
        print("testing")
        # 트레이닝한 model을 불러온다
        saver.restore(sess, model_path)
        print("from {} loaded".format(model_path))
        # label의 각 의미
        label_name_dict = {
            0: "jjajang",
            1: "tangsu"
        }
        # placeholder에 데이터를 넣는다.
        test_feed_dict = {
            datas_placeholder: datast,
            labels_placeholder: labelst,
            dropout_placeholdr: 0
        }
        predicted_labels_val = sess.run(predicted_labels, feed_dict=test_feed_dict)
        # 테스트결과와 실제결과를 비교한다.
        for fpathst, real_label, predicted_label in zip(fpathst, labelst, predicted_labels_val):
            real_label_name = label_name_dict[real_label]
            predicted_label_name = label_name_dict[predicted_label]
            print("{}\t{} => {}".format(fpathst, real_label_name, predicted_label_name))

Instructions for updating:
non-resource variables are not supported in the long term
shape of datas: (80, 32, 32, 3)	shape of labels: (80,)
shape of datas: (20, 32, 32, 3)	shape of labels: (20,)
2
Tensor("conv2d/Relu:0", shape=(?, 28, 28, 20), dtype=float32)
Tensor("max_pooling2d/MaxPool:0", shape=(?, 14, 14, 20), dtype=float32)
Tensor("conv2d_1/Relu:0", shape=(?, 11, 11, 40), dtype=float32)
Tensor("max_pooling2d_1/MaxPool:0", shape=(?, 5, 5, 40), dtype=float32)
Tensor("flatten/Reshape:0", shape=(?, 1000), dtype=float32)
Tensor("dense/Relu:0", shape=(?, 400), dtype=float32)
Tensor("dropout/Identity:0", shape=(?, 400), dtype=float32)
Tensor("dense_1/BiasAdd:0", shape=(?, 2), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

testing
INFO:tensorflow:Restoring parameters from ./models/tangjja.model


/Users/keunmo/Workspace/projects/chatbot/cbvenv/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/convolutional.py:414: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  warnings.warn('`tf.layers.conv2d` is deprecated and '
/Users/keunmo/Workspace/projects/chatbot/cbvenv/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/Users/keunmo/Workspace/projects/chatbot/cbvenv/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/pooling.py:310: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling2D` instead.
  warnings.warn('`tf.layers.max_pooling2d` is deprecated and '
/Users/keunmo/Workspace/p

from ./models/tangjja.model loaded
./data/tangjja_test/1_98.jpg	tangsu => tangsu
./data/tangjja_test/1_99.jpg	tangsu => tangsu
./data/tangjja_test/0_47.jpg	jjajang => jjajang
./data/tangjja_test/0_46.jpg	jjajang => jjajang
./data/tangjja_test/0_44.jpg	jjajang => jjajang
./data/tangjja_test/0_45.jpg	jjajang => jjajang
./data/tangjja_test/0_41.jpg	jjajang => jjajang
./data/tangjja_test/0_40.jpg	jjajang => jjajang
./data/tangjja_test/0_42.jpg	jjajang => jjajang
./data/tangjja_test/0_43.jpg	jjajang => tangsu
./data/tangjja_test/0_48.jpg	jjajang => jjajang
./data/tangjja_test/0_49.jpg	jjajang => jjajang
./data/tangjja_test/1_92.jpg	tangsu => tangsu
./data/tangjja_test/1_93.jpg	tangsu => tangsu
./data/tangjja_test/1_91.jpg	tangsu => tangsu
./data/tangjja_test/1_90.jpg	tangsu => tangsu
./data/tangjja_test/1_94.jpg	tangsu => tangsu
./data/tangjja_test/1_95.jpg	tangsu => tangsu
./data/tangjja_test/1_97.jpg	tangsu => tangsu
./data/tangjja_test/1_96.jpg	tangsu => tangsu
